In [1]:
import os
import sys
import shutil
import glob
import pandas as pd
import numpy as np
from scipy import stats
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from multiprocessing import Pool
from collections import Counter

In [7]:
# os.chdir("/data5/galaxy/project/lncRNA_analysis/enhancer")
mettle3_bed = "/data5/galaxy/project/mettl3_enrich/macs2_peak/mettl3.bed"
m6a_dir = "/data5/galaxy/project/promoter_TF_enrich/data/m6a_gene/promoter_bed"
m6a_list = glob.glob("%s/*.bed" % m6a_dir)
free_dir = "/data5/galaxy/project/promoter_TF_enrich/data/m6a_free/promoter_bed"
                # m6a-promoter  unm6a-promoter
# mettl3 peak         a             b
# non_peak            c             d
for m6a_bed in m6a_list:
    tissue = os.path.basename(m6a_bed)
    unm6a_bed = "%s/%s" % (free_dir, tissue)
    print(unm6a_bed)
    process_single_tissue(m6a_bed, unm6a_bed)

/data5/galaxy/project/promoter_TF_enrich/data/m6a_free/promoter_bed/liver.bed
1.8100265349850297 4.3223844959901104e-23
/data5/galaxy/project/promoter_TF_enrich/data/m6a_free/promoter_bed/stomach.bed
2.021274238227147 2.8474464710648426e-33
/data5/galaxy/project/promoter_TF_enrich/data/m6a_free/promoter_bed/placenta.bed
2.0108904741921947 3.8192180830316633e-26
/data5/galaxy/project/promoter_TF_enrich/data/m6a_free/promoter_bed/lung.bed
1.9341109169088238 3.073688985637997e-29
/data5/galaxy/project/promoter_TF_enrich/data/m6a_free/promoter_bed/heart.bed
2.087721649468345 7.215119088526416e-36
/data5/galaxy/project/promoter_TF_enrich/data/m6a_free/promoter_bed/brain.bed
2.379292748265396 1.6606444778722438e-49
/data5/galaxy/project/promoter_TF_enrich/data/m6a_free/promoter_bed/muscle.bed
1.892747547632021 2.7463568523115824e-24
/data5/galaxy/project/promoter_TF_enrich/data/m6a_free/promoter_bed/kidney.bed
2.1168801911108774 2.835772590992631e-27


In [6]:
def process_single_tissue(m_bed, u_bed):
    a = stat_intersect_num(m_bed)
    b = stat_intersect_num(u_bed)
#     print(a, b)
    c = int(os.popen("wc -l %s" % m_bed).read().split()[0]) - a
    d = int(os.popen("wc -l %s" % u_bed).read().split()[0]) - b
    fisher(a, b, c, d)

In [2]:
def stat_intersect_num(in_bed):
#     command = "bedtools intersect -a %s -b %s -wa -F 0.5 | sort | uniq | wc -l" % (in_bed, mettle3_bed)
    command = "bedtools intersect -a %s -b %s -wb -F 0.5 | sort | uniq | wc -l" % (in_bed, mettle3_bed)
    sub_p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    number = int(sub_p.communicate()[0].decode("utf-8"))
    return number

In [4]:
def fisher(a, b, c, d):
    oddsratio, pvalue = stats.fisher_exact([[a, b], [c, d]], alternative="greater")
    print(oddsratio, pvalue)